In [2]:
import imdb
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [3]:
# Spec our vote threshold
vote_threshold = 15000

In [4]:
# Combine data from imdb
info_df = pd.read_csv('data/title.basics.tsv', sep='\t')
ratings_df = pd.read_csv('data/title.ratings.tsv', sep='\t')
all_df = info_df.merge(ratings_df, on='tconst')

In [5]:
# Select our films for the game
movie_df = all_df[all_df['titleType'] == 'movie'] # Movies only
clean_movie_df = movie_df[movie_df['isAdult'] == 0] # No porn

pop_movies_df = clean_movie_df[clean_movie_df['numVotes'] > vote_threshold] #Greater than vote threshold
old_pop_movies = pop_movies_df[pd.to_numeric(pop_movies_df['startYear']) < 2014] 
new_pop_movies = pop_movies_df[(pd.to_numeric(pop_movies_df['startYear']) >= 2014) & (pop_movies_df['numVotes'] >= vote_threshold*10)] # New movies need more votes (people love voting on new movies!)

selected_films_df = pd.concat([old_pop_movies, new_pop_movies]).reset_index(drop=True)
selected_films_df['tconst'] = selected_films_df['tconst'].str.lstrip('tt')

selected_films_df.tail(5)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
5405,5362988,movie,Wind River,Wind River,0,2017,\N,107,"Crime,Drama,Mystery",7.7,163098
5406,5442430,movie,Life,Life,0,2017,\N,104,"Horror,Sci-Fi,Thriller",6.6,177480
5407,5463162,movie,Deadpool 2,Deadpool 2,0,2018,\N,119,"Action,Adventure,Comedy",7.8,359407
5408,5580390,movie,The Shape of Water,The Shape of Water,0,2017,\N,123,"Adventure,Drama,Fantasy",7.3,295113
5409,6644200,movie,A Quiet Place,A Quiet Place,0,2018,\N,90,"Drama,Horror,Mystery",7.6,283376


In [ ]:
# Pull in specific release dates & poster URLS

ia = imdb.IMDb()

releaseDate = []
posterURL = []
for idx, mv in tqdm(selected_films_df.iterrows(), total=selected_films_df.shape[0]):
    movie = ia.get_movie(mv['tconst']);
    mv_list = movie.keys()
    if 'original air date' in mv_list:
        releaseDate.append(movie['original air date'])
    else: 
        releaseDate.append(mv['startYear'])
    if 'cover url' in mv_list:
        posterURL.append(movie['cover url'])
    else:
        posterURL.append('')

2019-03-16 13:47:49,828 CRITICAL [imdbpy] /Users/kathryncogert/Documents/Repos/moviechron/mcenv/lib/python3.6/site-packages/imdb/_exceptions.py:34: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'http://www.imdb.com/title/tt0107978/plotsummary', 'proxy': '', 'exception type': 'IOError', 'original exception': timeout('The read operation timed out',)},); kwds: {}
Traceback (most recent call last):
  File "/Users/kathryncogert/Documents/Repos/moviechron/mcenv/lib/python3.6/site-packages/imdb/parser/http/__init__.py", line 232, in retrieve_unicode
    content = response.read()
  File "/Users/kathryncogert/miniconda3/lib/python3.6/http/client.py", line 456, in read
    return self._readall_chunked()
  File "/Users/kathryncogert/miniconda3/lib/python3.6/http/client.py", line 563, in _readall_chunked
    chunk_left = self._get_chunk_left()
  File "/Users/kathryncogert/miniconda3/lib/python3.6/http/client.py", line 546, in _get_chunk_left
    chunk_left

In [ ]:
selected_films_df['releaseDate'] = releaseDate
selected_films_df['posterURL'] = posterURL
relDate=selected_films_df['releaseDate'].str.replace(repl='',pat=')').str.split(pat='(', expand=True)

selected_films_df['releaseDate'] = pd.to_datetime(relDate[0])
selected_films_df['releaseCountry']= relDate[1]


selected_films_df.to_csv('data/selected_films_hard.tsv', sep='\t')
selected_films_df.tail(5)